# PyOps Connector

In [ ]:
# Uncomment and run this cell to install PyOps
# !pip install pyops

In [ ]:
import pyops
import logging
import json

In [ ]:
# read config file first
with open('config.json', 'r', encoding="utf-8") as f:
    config = json.load(f)
print(config)

# PEPS
PEPS Opensearch descriptors are:
"https://peps.cnes.fr/resto/api/collections/{collection}/describe.xml"

In [ ]:
collection = "S2ST"

In [ ]:
def PEPS_descr_url(collection):
    """
    Build the Description URL for a collection in PEPS
    """
    descr_URL = config['PEPS_url']
    if collection in config['PEPS_collections']:
        descr_URL += collection
    descr_URL += "/describe.xml"
    return descr_URL

In [ ]:
descr_PEPS = PEPS_descr_url(collection)
print(descr_PEPS)

In [ ]:
logging.basicConfig(filename='example.log',level=logging.INFO)

# Client params
# specify either search_endpoint or description_xml_url
# type = 'collection' or 'results'
client = pyops.Client(description_xml_url=descr_PEPS, type='collection')


In [ ]:
# get the possible search parameters
client.search_params

### Querying the catalog

In [ ]:
client = pyops.Client(description_xml_url=descr_PEPS, type='collection')

param_names = client.search_param_names
params = client.search_params
input_params = {}

form_fields = {"tileid":  "31TCJ", "page": "1", 'maxRecords': '50', 'cloudCover': '[10,70]'}
for ff in form_fields:
    value = form_fields[ff]
    if value and ff in param_names:
        print(params[param_names[ff]]["full_tag"])
        input_params[params[param_names[ff]]["full_tag"]] = {"value": value}

# Get Raw Results
raw_results = client.search(params=input_params)
print("total number of results:", client.pagination['total_results'])
print("last page:", client.pagination['last'] )
form_fields = client.pagination['next']
while True:
    for ff in form_fields:
        value = form_fields[ff]
        if value and ff in param_names:
            input_params[params[param_names[ff]]["full_tag"]] = {"value": value}
    client = pyops.Client(description_xml_url=descr_PEPS, type='collection')
    new_raw_results = client.search(params=input_params)
    print(len(new_raw_results))
    assert raw_results[0] != new_raw_results[0]
    raw_results.extend(new_raw_results)
   
    if form_fields == client.pagination['last'] :
        break
    form_fields = client.pagination['next']
    input_params = {}


In [ ]:
# Filtering results
filtered_results = client.filter_entries([{
    "tag": "{http://www.w3.org/2005/Atom}title",
    "name": "title"}, {
    "tag": "{http://www.w3.org/2005/Atom}summary",
    "name": "summary"}])
for results in filtered_results:
    for res in results:
        print(res)